## 4.1 Modélisation de l'héritage avec une table unique

### Implémentation sous SQLAlchemy

In [1]:
from sqlalchemy import Table, Column,  ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Moteur d'accès à la base sqlalchemy.sqlite
engine = create_engine('sqlite:///sqlalchemy.sqlite')

In [2]:
# Session
DBSession = sessionmaker(bind=engine)
session = DBSession()

<p id="declarations" style="font-size:120%">Déclaration des classes métier :</p>

In [3]:
# Classe mère
Base = declarative_base()
Base.metadata.bind = engine

# Définition des classes 
class Person(Base):
    __tablename__ = 'person'

    id = Column(Integer, primary_key=True)
    first_name = Column(String(64), nullable=False)
    last_name = Column(String(64), nullable=False)
    type = Column(String(20), nullable=False)

    __mapper_args__ = {
        "polymorphic_on": type,
        "polymorphic_identity": "person",
    }
    
    def __str__(self):
        return '{} {} {} {}'.format(self.id,self.type,self.first_name,self.last_name)

class Developer(Person):
    login = Column(String(32))
    pwd = Column(String(32)) 

    __mapper_args__ = {
        "polymorphic_identity": "developer",
    }

    def __str__(self):
        return super().__str__() + ' {} {}'.format(self.login,self.pwd)

class Player(Person):
    pseudo = Column(String(16))
    score = Column(Integer)
    
    __mapper_args__ = {
        "polymorphic_identity": "player",
    }

    def __str__(self):
        return super().__str__() + ' {} {}'.format(self.pseudo,self.score)

class VIP(Player):
    email = Column(String(255))
    preferences = Column(String(255)) 

    __mapper_args__ = {
        "polymorphic_identity": "VIP",
    }

    def __str__(self):
        return super().__str__() + ' {} {}'.format(self.email,self.preferences)


# Création éventuelle des tables dans la base
Base.metadata.create_all(engine)

<div id="usage"/>
#### Classe Person : exemple d'usage

In [4]:
# Création éventuelle d'une personne
people = session.query(Person).filter(Person.id == 1).all()
if len(people) == 0:
    person = Person(first_name="Raymond",last_name='Deubaze')
    session.add(person)
    session.commit()
else:
    person = people[0]

print(person)

1 person Raymond Deubaze


#### Classe Player : exemple d'usage

In [5]:
# Création éventuelle d'un player
players = session.query(Player).all()
if len(players) == 0:
    player = Player(first_name="Jean",last_name='Bonnot',pseudo="jbt",score=0)
    session.add(player)
    session.commit()
else:
    player = players[0]

print(player)

2 player Jean Bonnot jbt 0


#### Classe Developer : exemple d'usage

In [6]:
# Création éventuelle d'un developer
devs = session.query(Developer).all()
if len(devs) == 0:
    dev = Developer(first_name="Maud",last_name='Zarella',login="mozarella",pwd='di Bufala')
    session.add(dev)
    session.commit()
else:
    dev = devs[0]

print(dev)

3 developer Maud Zarella mozarella di Bufala


#### Classe VIP : exemple d'usage

In [7]:
# Création éventuelle d'un VIP
vips = session.query(VIP).all()
if len(vips) == 0:
    vip = VIP(first_name="Kelly",last_name='Diotte',pseudo="kde",score=0,email="kelly@gmail.com")
    session.add(vip)
    session.commit()
else:
    vip = vips[0]

print(vip)

4 VIP Kelly Diotte kde 0 kelly@gmail.com None


<div id="liste_polymorphique"/>
#### Requête renvoyant une liste polymorphique

In [8]:
# Requête all sur la table des users
all_people = session.query(Person).all()
for p in all_people:
    print("{0: <10} ".format(p.__class__.__name__) + str(p))

Person     1 person Raymond Deubaze
Player     2 player Jean Bonnot jbt 0
Developer  3 developer Maud Zarella mozarella di Bufala
VIP        4 VIP Kelly Diotte kde 0 kelly@gmail.com None
